## Introduction to Sagemaker Automatic Model Tuning
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. You can use Amazon SageMaker automatic model tuning with built-in algorithms, custom algorithms, and Amazon SageMaker pre-built containers for machine learning frameworks.
For more details on Sagemaker automatic model tuning, please refer to the following link:
https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html


## Tuning our clustering model
We have been able to determine that the drugs provided to us can be clustered into 3 different cluster groups based on just the review text provided to us. However, we now want to determine the appropriate number of clusters for our data. To do this, we choose the [silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) analysis method.
Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].
Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.
For more details about Silhouette analysis method, please refer to the following link:
https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

## Setup
Lets begin by importing the necessary libraries and packages we will be using. Note the new [`sagemaker.tuner`](https://sagemaker.readthedocs.io/en/stable/tuner.html) class that will be used for our analysis.

In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import boto3
import pandas as pd
import time
from sagemaker.tuner import IntegerParameter, HyperparameterTuner

Lets now get the `sagemaker_session_variable`, `bucket`, `role`, and the `train_input` data path.

In [ ]:
sagemaker_session_variable=sagemaker.Session()
bucket = sagemaker_session_variable.default_bucket()

role = get_execution_role()
#prefix = 'sagemaker/ae-clustering'

train_input = "s3://{}/scikit-kmeans/data/vectors.csv".format(bucket)
print (train_input)

## Training Script for automatic model tuning
As mentioned earlier, we will be using sagemaker automatic model tuning to get the maximum value of the metric silhouette-coefficient. The model that provides the maximum value of this metric is the best model. This will allow us to determine the exact number of clusters we should use for our dataset. To achive this, we will use a training script that calculates the value of silhouette-coefficient and print it in a certain format. We will then define a regex expression in the metric definition to catch the value of the metric. 
<br><br>Let's look at the script in more detail.


In [ ]:
! cat condition_cluster_silhouette_hpo.py

The script will look very similar to the script we ran in the previous notebook except we are now normalizing our training data and printing the value of silhoutte coefficient. 
<br>
We will now create our estimator with the new script just like we did in the previous notebook.

In [ ]:
script_path = 'condition_cluster_silhouette_hpo.py'

sklearn = SKLearn(
    entry_point=script_path,
    train_instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session_variable)

Next, we define the objective metric name, metric definition as a regex and the range of hyperparameters we want to tune. In our case, this parameter is the number of clusters. By default, SageMaker automatic model tuning will maximize the objective metric value.

In [ ]:
objective_metric_name = 'silhouette-coefficient'
metric_definitions = [{'Name': 'silhouette-coefficient',
                       'Regex': "silhouette-coefficient = (.*?);"}]

hyperparameter_ranges = {'n_clusters': IntegerParameter(2,10)}

Now, we'll create a `HyperparameterTuner object`, to which we pass:

* The Scikitlearn estimator we created above
* Our hyperparameter ranges
* Objective metric name and definition
* Number of training jobs to run in total and how many training jobs should be run simultaneously.
* Whether we should maximize or minimize our objective metric (we haven't specified here since it defaults to 'Maximize', which is what we want for silhouette-coefficient)

In [ ]:
tuner = HyperparameterTuner(sklearn,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3)

The below fit method is run asynchronously, so the cell will complete execution immediately. If you want to run it to wait, you can set the boolean attribute "wait" as True.

In [ ]:
tuner.fit({'train': train_input}, wait=False)

You can check the status of this job by calling the describe_hyper_parameter_tuning_job API as shown below. Once all of the 9 jobs have completed, the status will change to "Complete". Please wait till all jobs complete.

In [ ]:
while True:
    status = boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']
    print (status)
    if status == 'Completed': break
    time.sleep(60)


**Running all 9 jobs takes time.** An easy way to look at the status of the jobs is in the Sagemaker console. You can view it at the follwoing link:
 https://console.aws.amazon.com/sagemaker/home?region=us-east-2#/hyper-tuning-jobs
 
 You should see an in progress job. Click on it to see the details of the individual model runs. Recall that we chose to run a total of 9 jobs with a parallelization of 3 so the process will complete when our training has run 3 times. **Wait for it to complete before moving forward.** 

## Analyzing the results of the tuning job
SageMaker provides a handy [`HyperparameterTuningJobAnalytics`](https://sagemaker.readthedocs.io/en/stable/analytics.html) class that allows you to import the results of the tuning job into a dataframe for analysis. There are detailed examples of the full feature set of sagemaker automatic model tuning here: https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning 
<br><br>
The code below loads the results of the tuning jobs into a dataframe for analysis. You can see the lowest and the highest value of the objective metric (silhouette-coefficient) and other details like the training time, the parameters used and job status.

In [ ]:
analysis = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name)

full_df = analysis.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Next, we will show how the objective metric changes over time, as the tuning job progresses. For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance exploration of new areas of parameter space against exploitation of known good areas. This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.analysis = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.analysis.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(analysis)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [ ]:
ranges = analysis.tuning_ranges
figures = []
tuning_job_result = boto3.Session().client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)

objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))

Based on the graph above, you can see that as the number of clusters increase, the value of silhouette-coefficient increases till we reach a max value at 3 clusters. Post that, the value of silhouette-coefficient decreases. This could vary for each run of the tuning job so check out what the optimal number of clusters are for your case using the code below.

In [ ]:
best_n_clusters=int(full_df.loc[full_df['FinalObjectiveValue'].idxmax()]['n_clusters'])
best_n_clusters

## Conclusion
Its important to note that there are other ways to evaluate the performance of a clustering model like [homogenity score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html#sklearn.metrics.homogeneity_score) and [completeness score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html#sklearn.metrics.completeness_score). However, these methods require the availability of ground truth labels. Since we do not have that, silhouette-coefficient is an appropriate way to evaluate our model. As a next step, we can treat the labels generated by our clustering model as ground truth and evalutate some of the other metrics to see how our model performs.
<br><br>
We have now concluded our workshop. We hope you found the content valuable. **Thank you for your participation!**